# Is there a Vélib station nearby ?

Application of [OAlley API](https://api.oalley.fr) to create a map of coverage area of Vélib stations (Paris city bike rental).

Algorithm:
1. Fetch a list of all GPS coordinates of Velo Toulouse stations.
2. For each station, compute a 5 minutes by foot isochrone with [OAlley API](https://api.oalley.fr) .
3. Using `shapely`, perform an union on all requested zones to compute the final zone.
4. Draw with folium

In [ ]:
import grequests
import requests
import polyline
from urllib.parse import urlencode
import json

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"

## API calls

Fetch the list of stations then make all requests to OAlley API and save to JSON


In [2]:
# Request bike stations
r = requests.get("http://data.iledefrance.fr/explore/dataset/velib_a_paris_et_communes_limitrophes/download?format=json")

# Extract coords for each
coords = [{'lat': s['geometry']['coordinates'][1], 'lng': s['geometry']['coordinates'][0], 'duration': 0} for s in r.json()]

coords_5m = [dict(c, duration = 60) for c in coords]

print("Found %d stations" % len(coords))

zones = []

# Called when OAlley returned a zone
def callback(res, **kwargs):
    global i
    if(res.status_code != 200): 
        return print(res.json()) 
    
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    

# Use a session to avoid watching over too many sockets
session = requests.Session()

# Prepare all requests for OAlley
urls = [baseurl + appkey + method + urlencode(point) for point in coords_5m]
reqs = [grequests.get(url, callback=callback, session=session) for url in urls]


# Execute all first 400 requests (because OAlley API is limited during beta phase)
grequests.map(reqs[0:400], exception_handler=exception_handler)
print("400 / %d " % len(urls))

# 400 next
grequests.map(reqs[400:800], exception_handler=exception_handler)
print("800 / %d " % len(urls))

# all remaining
grequests.map(reqs[800:], exception_handler=exception_handler)
print("Saving to file...")

filename = 'paris_velib_zones.json'

with open(filename, 'w+') as f:
    json.dump(zones, f)
    
print("Saved to %s " % filename)

Found 1229 stations
400 / 1229 
800 / 1229 
Done !


In [6]:
import folium
import json
from shapely.geometry.polygon import Polygon
from shapely.ops import cascaded_union
import warnings

# Read back the file 
# Executing previous cells is optional, only required if the data does not need to be updated
zones = []

with open('paris_velib_zones.json', 'r') as f:
    zones = json.load(f)

In [7]:
# Merge all zones

polygons = [Polygon(z) for z in zones]

nonvalids = [p for p in polygons if not p.is_valid]

# Some zones have issues for shapely. Ignore them.
print("Found %d invalid polygons" % len(nonvalids))
    
polygons = [p for p in polygons if p.is_valid]
    
allzones = cascaded_union(polygons)

Self-intersection at or near point 48.876959999999983 2.3664500000000412
Self-intersection at or near point 48.875688358208969 2.3565974626865529
Self-intersection at or near point 48.868684797280892 2.3234786744355422
Self-intersection at or near point 48.876040000000003 2.2966799999999998
Self-intersection at or near point 48.866590000000002 2.3077899999999998
Self-intersection at or near point 48.863716154323541 2.3104084215970211
Self-intersection at or near point 48.856997455357138 2.3011276339285791
Self-intersection at or near point 48.851939999999999 2.3431000000000002
Self-intersection at or near point 48.844458294077292 2.3244475623142016
Self-intersection at or near point 48.859480000000005 2.3576842857142797
Self-intersection at or near point 48.842828173054848 2.3553649185219196
Self-intersection at or near point 48.850714818921787 2.363284971098266
Self-intersection at or near point 48.854929654605264 2.3554885074013217
Self-intersection at or near point 48.85966745098038

Found 302 invalid polygons


Self-intersection at or near point 48.880269454170964 2.337448434603504
Self-intersection at or near point 48.844346734403381 2.3362756864477494
Self-intersection at or near point 48.827643145346173 2.3336935167354427
Self-intersection at or near point 48.819836971713812 2.3569777337770397
Self-intersection at or near point 48.863959999999999 2.3494799999999998
Self-intersection at or near point 48.842976883677878 2.3552978093181127
Self-intersection at or near point 48.868306842105241 2.3063105263157846
Self-intersection at or near point 48.868498435127101 2.3663678441729932
Self-intersection at or near point 48.826994748858446 2.3381160512209664
Self-intersection at or near point 48.832569279778397 2.3309365571824294
Self-intersection at or near point 48.833181000000081 2.3743459999999228
Self-intersection at or near point 48.835749999999997 2.3236500000000002
Self-intersection at or near point 48.847699999999968 2.2897699999999666
Self-intersection at or near point 48.86358582857143

Merged 927 into 8


In [8]:
print("Merged %d into %d" % (len(polygons),len(allzones)))

Merged 927 into 8


# Is there a Vélib station near you ? The map.

Area in blue represents where you can find a Vélib station (Paris city bike) for less than 5 minutes by walk.

Made using [OAlley API](https://api.oalley.fr)

# Une station de Vélib à moins de 5 minutes a pied ? Voici la carte.

La zone en bleu représente la zone ou vous pouvez trouver une station de Vélib à Paris à moins de 5 minutes a pied (approximativement).

Réalisé avec [l'API OAlley](https://api.oalley.fr)

In [9]:
# Build output map
m = folium.Map(location=[46, 2], zoom_start=5)

contour_color = "#ffffff00" 
fill_color    = "#479eff"
fill_color2    = "#03005f"

# Draw all computed zones on the map
for zone in allzones:
    pts = (list(zone.exterior.coords))
    folium.features.PolygonMarker(locations=pts, color=contour_color, fill_color=fill_color,  weight=1).add_to(m)

# Print the result
m.fit_bounds(m.get_bounds())

# IPython trick to display the map
m

In [6]:
# Eventually, dump all zones to a google polyline format if you want to display it on a website
arr = [ polyline.encode(list(z.exterior.coords)) for z in allzones]

json.dumps(arr)

'["ehdiGuhpGcAiCcAbD]nCdAfEQ@M?eLy@oLxBCDhBkPScGhB`A\\\\E|GTfHqEwCq@kGTlG_JBwEHoFcPjPu@aAeAqCmBnI]AeEu@uB`AUHz@fCFnAuGdClCdD@?_DzL|B|B~EGbBxFhEp@PyAxON|BN|@d@jC_Ad@}B|@zC|@zGtEdJv@gGtJvHfDwDjF}@cPgMhDyBqBeCZqAgDmBk@aATgCdDcQ{IjM}Ai@yBhC", "wkfiGs~oGi@mCqU}n@~Ixe@uK~@|EtB[xAwHJ`AlIi@xDoCjDcBtJf_@aT?N`AmAXAZA\\\\?X?r@J@@D?HCHQjLQqLi@vBcBC@kCnA_@By@y@ESEO", "{meiG}fsG{GuMaB~FyAnCdAbIkAHzEdGFj@qIjE_JrMVdFpHwBxGdCj@vCrHmBfBwBbCoCn@_DvBoAn@s@oA{DQ]fI_Cu@yA^uE`@mD{C{BuAk@wEx@uCUmBO", "ax_iGqplG{@pAiBjC_@n@a@j@yVaJXxC}C~HaAZ@@JHRLZP??VP`AbIB`DnLgDd@dKPAxACfJlCxEdDtBxAnChB`KARsJbG}@dAXcDwQj@mB`@{@`Au@vA}@f@aAlBc@jGcD|GcEeOMlA_Iw@UqCN}CS\\\\eIkAGxAkGiBkMRs@x@AzBnMvHuQn@VvQfOiEiSoCqJpD_Dc@kB[wBfPgGoD}EkCaCwBp@qQqEd@aHkBhGw`@_KnCoADAti@}Rt@j@qBzAzCZb@j@h@?}ArJ?jTxF{B`Fq@b@aHp@yIdCiBXe@vDt@_C_GFoBcGk@lBsBnAwCeDHUQMe@dDeTcGxEiD|AoBkEHuAKC_@Mi@iAx@eCzAqHNOpAkH|FyAzAd@lA\\\\hAwAqA{DlBaCB@MGeC_B@Yn@c@f@yCf@kAlHpGgBlCYPvHjBGFmElE`AvAfAzAnBb@bCaBrAlCjD~FfCe@{AxC`A~AzBhAlBsF|ElBRv@?CJq@`GzG?eG{AcE\\\\mAqA